In [2]:
import numpy as np
import matplotlib.pyplot as plt

# FFT for option pricing

talk in the motivation that it is interesting to use this method because computationally is very good, complexity is N log N instead of N2

We would like to dive deeper in the study of option price of assets under the Heston model. Since we can find analytically the characteristic function of the risk-neutral density of such assets, we can use the proposed FFT method for option pricing. Here we will study the pricing of an European call of maturity $T$, hence with payoff:

$$
\text{Payoff} = (S_{T} - K)^{+}
$$

Where $S_{T}$ is the price at maturity of the given asset. In the following we will keep the same notation as in the paper. 

Let us look at the risk-neutral density of the log price $s_{T} = \log(S_{T})$ of density $q_{T}$. The characteristic function of the log price is therefore:

$$
\phi_{T}(u) = \int_{-\infty}^{\infty} e^{ius}q_{T}(s)ds
$$

Under the risk-neutral assumption, we have that the price $C_{T}(k)$ of an European call with strike log-strike $k$ ($k = \log(K)$) and maturity $T$:

$$
C_{T}(k) = \int_{k}^{+\infty} e^{-rT} (e^{s} - e^{k}) q_{T}(s)ds
$$

Since when $k\rightarrow - \infty$ we have that $C_{T}(k)\rightarrow S_{0}$, the price of the call is not square-integrable, and thus we perform the following adaptation:

$$
c_{T}(k) := e^{\alpha k} C_{T}(k)
$$

Which, for a certain range of values of $\alpha$ that will be determined later, makes $c_{T}(k)$ a square integrable function in $k$. Now let us perform the Fourier transform of $c_{T}(k)$:

$$
\Psi(v) = \int_{-\infty}^{+\infty} e^{ivk}c_{T}(k)dk
$$

Using the inverse Fourier transform, we can retrieve the call price:

$$
C_{T}(k) = \frac{e^{- \alpha k}}{2\pi}\int_{-\infty}^{\infty}e^{-ivk}\Psi_{T}(v)dv
$$

Since $C_{T}(k)$ is real:

$$
C_{T}(k) = \frac{e^{- \alpha k}}{\pi}\int_{0}^{\infty}e^{-ivk}\Psi_{T}(v)dv
$$

Now if we look back at the forward Fourier transform, may compute it explicitely:

$$
\Psi_{T}(v) = \int_{-\infty}^{\infty} e^{ivk} \int_{k}^{+\infty}e^{\alpha k} e^{-rT}(e^{s} - e^{k})q_{T}(s) dsdk
$$

Inversing the integrals:

$$
\Psi_{T}(v) = \int_{-\infty}^{\infty} e^{-rT} q_{T}(s)\int_{-\infty}^{s}(e^{s+\alpha k} - e^{1+\alpha}k)e^{ivk}dkds
$$

$$
\Psi_{T}(v) = \int_{-\infty}^{\infty} e^{-rT} q_{T}(s) \left(\frac{e^{(\alpha + 1 + iv)s}}{\alpha + iv} - \frac{e^{(\alpha + 1 + iv)s}}{\alpha + 1 + iv}\right) ds
$$

$$
\Psi_{T}(v) = \frac{e^{-rT}}{\alpha^{2} + \alpha - v^{2} + i(2\alpha + 1)v}   \int_{-\infty}^{\infty} e^{i(v -(\alpha + 1)i)s} q_{T}(s) ds
$$

Which is exactly, by using the characteristic function of the log-price:

$$
\Psi_{T}(v) = \frac{e^{-rT} \phi_{T}(v - (\alpha + 1)i)}{\alpha^{2} + \alpha - v^{2} + i(2\alpha + 1)v} 
$$

Hence, we could replace that on our equation for $C_{T}(k)$:

$$
C_{T}(k) = \frac{e^{- \alpha k}}{\pi}\int_{0}^{\infty}e^{-ivk}\Psi_{T}(v)dv
$$

To obtain the price of our call. We remark that this is simply a direct Fourier transform. As indicated in the paper, we remark that the denominator vanishes when both $\alpha = 0$ and $v = 0$, inducing a singularity, which reinforces the choice of adding $\alpha$ for integrability purposes.

## Adressing the choice of $\alpha$

## Option pricing using FFT

Now we move to the numerical part of the problem. Here we attempt to rewrite the integration of the previous section as a summation that allows direct application of the FFT.

Set $v_{j} = \eta(j-1)$. Using the trapezoid rule on the integral:

$$
C_{T}(k) \approx \frac{e^{-\alpha k}}{\pi} \sum_{j=1}^{N} e^{-iv_{j}k}\Psi_{T}(v_{j})\eta
$$

Which we do by setting the upper limit of our integral at $a = N\eta$

In other words, we simply fixed $a$ to be the limit of our integral and created a grid of with $N$ steps in our summation.

Since we are interest in at-the-money call values we are looking at values of $k$ that are near 0.

The FFT shall return $N$ values of $k$ to which we employ a regular spacing of size $\lambda$, hence, our values of $k$ are:

$$
k_{u} = -b + \lambda (u-1), \quad u\in \{1,\dots, N\}
$$

We now replace substitute these values on our summation:

$$
C_{T}(k_{u}) \approx \frac{e^{-\alpha k_{u}}}{\pi} \sum_{j=1}^{N} e^{-iv_{j}k_{u}}\Psi_{T}(v_{j})\eta
$$

$$
C_{T}(k_{u}) \approx \frac{e^{-\alpha k_{u}}}{\pi} \sum_{j=1}^{N} e^{-iv_{j}[-b+\lambda (u-1)]}\Psi_{T}(v_{j})\eta
$$

$$
C_{T}(k_{u}) \approx \frac{e^{-\alpha k_{u}}}{\pi} \sum_{j=1}^{N} e^{-iv_{j}\lambda (u-1)}e^{ibv_{j}}\Psi_{T}(v_{j})\eta
$$

We can also replace $v_{j}$:

$$
C_{T}(k_{u}) \approx \frac{e^{-\alpha k_{u}}}{\pi} \sum_{j=1}^{N} e^{-i\eta\lambda(j-1)(u-1)}e^{ibv_{j}}\Psi_{T}(v_{j})\eta
$$

To apply the fast Fourier transform we must guarantee therefore:

$$
\lambda \eta = \frac{2\pi}{N}
$$

Hence there is a clear trade off between having a fine grid with $\eta$ small, and having a small space between the values of $k_{u}$ that we are able to compute.

To be able to maintain accurate enough integration with larger values of $\eta$ we can incorporate Simpson's rule (composite Simpson's 1/3 rule), rewriting our summation as:

$$
C_{T}(k_{u}) \approx \frac{e^{-\alpha k_{u}}}{\pi} \sum_{j=1}^{N} e^{-i\frac{2\pi}{N}(j-1)(u-1)}e^{ibv_{j}}\Psi_{T}(v_{j})\frac{\eta}{3}[3+(-1)^{j} - \delta_{j-1}]
$$

It is our task to therefore choose appropriately $\lambda$ and $\alpha$

## Computing the Heston characteristic function $\phi$

The heston stochastic model is given by the system of SDE :

$$dS_t = \mu S_t dt + \sqrt{V_t} S_t dW_S(t)$$
$$dV_t = \kappa (\theta - V_t) dt + \omega \sqrt{V_t} d W_V(t)$$

with correlated Brownian motions $dW_S(t)dW_V(t) =  \rho dt$, and two initial state $S_0$ and $V_0$.

We recall the characteristic function of $S_t$ : $$\phi(u) = \mathbb{E}[S_t^{iu}] = \mathbb{E}[e^{iu \ln(S_T)}] $$

You can find the computation of the characteristic function in the original paper of Heston, and it's defined as follow :

$$\phi(u) = e^{C(u) + D(u)V_0 + iu \ln F} $$

Where

$$F =S_0 e^{\mu T}$$

$$C(u) = \frac{\kappa \theta}{\omega^2} \left( (\kappa - \rho \omega u i + d(u))T - 2 \ln \left(\frac{c(u)e^{d(u)T} - 1}{c(u) - 1} \right) \right)$$

$$D(u) = \frac{\kappa - \rho \omega u i + d(u)}{\omega^2} \cdot \frac{e^{d(u)T} - 1}{c(u)e^{d(u)T} - 1}$$


with 

$$ c(u) = \frac{\kappa - \rho \omega u i + d(u)}{\kappa - \rho \omega u i - d(u)}, \ \  \ \  \ \   d(u)  = \sqrt{(\rho \omega u i - \kappa)^2 + iu\omega^2 + \omega^2 u^2}$$ 

One can notice that these equations make use of the logarithm of complex numbers, which is a function that is could have multiplied definition, and not continuous.

for example, if $z = x + iy =  r_z e^{i t_z} $ where $ t_z \in ]-\pi, \pi] $, the logarithm of $z$ in python liberaries is : 

$$log(z) = log(r_z) + i t_z$$

Called the principal branche.


- We studied the paper of C.Kahl, P.Jackel ["Not-so-complex logarithms in the Heston model"], where the authors pointed out the errors of option pricing involving Heston stochastics model due to the evaluation of complex argument.
- The paper proposes an alternative to solve this issue, and that garanties the continuity of the $\phi(u)$

First of all, we write in another way the characteristic function :

$$\phi(u) = G(u)^{-2\gamma} e^{R(u) + D(u)V_0 + iu \ln F} $$

where $$G(u) = \frac{c(u)e^{d(u)T} - 1}{c(u)-1}, \ \   R(u) = \gamma (\kappa - \rho \omega u i + d(u))T,  \ \  \gamma = \frac{\kappa \theta}{\omega^2}$$

As such $C(u) = R(u) - 2\gamma \ln G(u)$



Now we reduce the problem on the computation of $\ln G(u)$

To do that the paper introduces these following quantities :
 
$$ c = r_c e^{it_c}$$
$$ d = a_d + i b_d$$
$$ c - 1 =  r^* e^{i(\chi^* + 2\pi m)}$$
where $ m = [(t_c + \phi) / (2 \pi)] $, $\chi^* = arg(c-1)$ and $r^* = |c-1|$

$$ce^{dT} - 1 = r^{**}e^{i(\chi^{**} + 2\pi n)}$$
where $ n = [(t_c + b_d T + \pi) / (2\pi)] $, $\chi^* = arg(ce^{dT} - 1)$ and $r^* = |ce^{dT} - 1|$


with these defined variables we obtain :

$$G(u) = \frac{ce^{dT} - 1}{c-1} = \frac{r^*}{r^{**}} e^{\chi^{**} - \chi^{*} + 2\pi(n-m)}$$


Hence the logarithm of $G(u)$ can be expressed as :

$$ \ln G(u) = \ln (\frac{r^*}{r^{**}}) + i [\chi^{**} - \chi^{*} + 2\pi(n-m)]$$

In [4]:
# Implementation
def phi(S0, V0,  T, u,  mu, kappa, theta, omega, rho):
    """returns the characteristic function of a heston model on a maturity T
    and on the point u

    Args:
        S0 (float): the initial value of the asset
        V0 (float): the initial value of squared volatility
        T (float): the maturity
        u (float): the point of evaluation of phi 
        @parameters of the Heston model
        mu (float)
        kappa (float)
        theta (float)
        omega (float)
        rho (float)
    """

    # Define F
    
    F = S0 * np.exp(mu * T)
    
    # Complex unit i

    i = complex(0, 1)

    # Line 1: Calculate d

    d = np.sqrt((rho * omega * u * i - kappa)**2 + omega**2 * (u*i + u**2))

    # Line 2: Calculate c

    c = (kappa - rho * omega * u * i + d) / (kappa - rho * omega * u * i - d)

    # Line 3: Calculate tc (argument of c)

    tc = np.angle(c)

    # Line 4: Calculate GD

    GD = c - 1

    # Line 5: Calculate m

    m = int((tc + np.pi) / (2 * np.pi))

    # Line 6: Calculate GN

    GN = np.exp(d * theta) - 1

    # Line 7: Calculate n

    n = int((tc + np.imag(d) * theta + np.pi) / (2 * np.pi))

    # Line 8: Calculate lnG

    lnG = np.log(np.abs(GN) / np.abs(GD)) + i * (np.angle(GN) - np.angle(GD) + 2 * np.pi * (n - m))

    # Line 9: Calculate D

    D = (rho - rho * omega * u * i + d) / (omega**2 * (np.exp(d * theta) - 1))

    # Line 10: Calculate C

    C = (omega**2 / (kappa - rho * omega * u * i + d) * (np.exp(d * theta) - 1)) - 2 * lnG

    # Line 11: Calculate phi

    phi = np.exp(C + D * V0 + i * omega * u * np.log(F))

    return phi


In [3]:
# # Generate sample data
# sample_rate = 1000  # Number of samples per second
# duration = 1  # Duration in seconds
# t = np.arange(0, duration, 1/sample_rate)  # Time array
# frequency1 = 50  # Frequency of the first sine wave in Hz
# frequency2 = 120  # Frequency of the second sine wave in Hz
# signal = np.sin(2 * np.pi * frequency1 * t) + 0.5 * np.sin(2 * np.pi * frequency2 * t)

# # Perform FFT
# fft_result = np.fft.fft(signal)
# frequencies = np.fft.fftfreq(len(fft_result), 1/sample_rate)

# # Plot the original signal and its FFT
# plt.subplot(2, 1, 1)
# plt.plot(t, signal)
# plt.title('Original Signal')

# plt.subplot(2, 1, 2)
# plt.plot(frequencies, np.abs(fft_result))
# plt.title('FFT Result')
# plt.xlabel('Frequency (Hz)')

# plt.tight_layout()
# plt.show()